In [16]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [17]:
Rat = np.array([[4,5,6,7,8,9],[3,4,6,7,1,9],[2,5,6,8,9,10]],dtype=float).T #this is the ratings matrix
Rat = torch.from_numpy(Rat) #import torch to make Ratings a tensor
print("The ratings Matrix is = ",Rat) # 5,3

#Random initialization of Machine Learning Parameters
X = torch.randn((6,4),dtype = float,requires_grad = True)
theta = torch.randn((4,3),dtype =float, requires_grad = True)
#make predictions
preds = X @ theta 
loss_fn = F.mse_loss
#set Loss function ot be the Mean Squared Error
weight_decay = 1e-5
# Add regularization terms to the loss function
regex = (torch.sum(torch.square(X)))*weight_decay
regtheta = (torch.sum(torch.square(theta)))*weight_decay
print(regex)
loss = loss_fn((X @ theta), Rat) + regex + regtheta
print("loss = ",loss) # use loss.item()
print(X,"\n",theta)


The ratings Matrix is =  tensor([[ 4.,  3.,  2.],
        [ 5.,  4.,  5.],
        [ 6.,  6.,  6.],
        [ 7.,  7.,  8.],
        [ 8.,  1.,  9.],
        [ 9.,  9., 10.]], dtype=torch.float64)
tensor(0.0002, dtype=torch.float64, grad_fn=<MulBackward0>)
loss =  tensor(49.3705, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[ 0.1336,  0.3797, -0.4243,  1.8808],
        [ 0.1657,  0.8435, -0.5216, -0.1533],
        [ 0.4720, -1.0782, -0.6987,  1.3342],
        [-0.5224,  0.4308, -0.1668, -0.2592],
        [-0.0223, -0.1947,  0.5834,  2.1732],
        [-1.0746, -0.9070, -0.7825, -1.0983]], dtype=torch.float64,
       requires_grad=True) 
 tensor([[-0.6319, -0.2473,  1.3060],
        [-0.7780,  2.2813, -0.0416],
        [-0.2793,  2.0412,  0.0669],
        [ 0.6066, -0.9522,  0.2399]], dtype=torch.float64, requires_grad=True)


In [18]:
#we are taking full batch size (not batch Gradient Descent)! 
batch_size = 5;
train_ds = TensorDataset(Rat) #wrap into a TensorDataset
train_dl = DataLoader(train_ds,batch_size, shuffle = True)
for rat in train_dl:
  print(rat)
  break

[tensor([[4., 3., 2.],
        [8., 1., 9.],
        [7., 7., 8.],
        [6., 6., 6.],
        [5., 4., 5.]], dtype=torch.float64)]


In [19]:
rat = Rat.clone().detach()
print("This is a copy of Ratings ",rat)

This is a copy of Ratings  tensor([[ 4.,  3.,  2.],
        [ 5.,  4.,  5.],
        [ 6.,  6.,  6.],
        [ 7.,  7.,  8.],
        [ 8.,  1.,  9.],
        [ 9.,  9., 10.]], dtype=torch.float64)


In [20]:
##-------------------this is MY own gradient descent Implementation , this works as well -------------
# num_epochs = 100000
# learning_rate  = 1e-3;
# for i in range (num_epochs):
#   rat = X @ theta;
#   loss = loss_fn(rat, Rat);
#   loss.backward();
#   with torch.no_grad():
#     theta -= theta.grad*learning_rate
#     X -= X.grad*learning_rate
#     X.grad.zero_()
#     theta.grad.zero_()
#   #the variables in this system are X, theta and rat ( ratings ) 
  
# print(loss_fn((X @ theta),Rat), '\n')
# print("Predicted Ratings = \n", (X @ theta))
# print("Ratings =\n ",Rat)
# print(X)
# print(theta)


In [21]:

optimizer = torch.optim.SGD((X,theta), lr = 1e-3 , weight_decay = 1e-5)
for i in range (10000):
  for r in train_dl:
    loss = loss_fn(X @ theta, Rat)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()



In [22]:
print("The predicted Ratings after minimizing the Loss function = \n ",X @ theta)

The predicted Ratings after minimizing the Loss function = 
  tensor([[ 3.9243,  2.9970,  2.0720],
        [ 5.1464,  4.0058,  4.8606],
        [ 5.9547,  5.9982,  6.0431],
        [ 7.0817,  7.0032,  7.9222],
        [ 7.9779,  0.9992,  9.0210],
        [ 8.9308,  8.9972, 10.0658]], dtype=torch.float64,
       grad_fn=<MmBackward0>)


In [23]:
print("The real Ratings = \n ",Rat)

The real Ratings = 
  tensor([[ 4.,  3.,  2.],
        [ 5.,  4.,  5.],
        [ 6.,  6.,  6.],
        [ 7.,  7.,  8.],
        [ 8.,  1.,  9.],
        [ 9.,  9., 10.]], dtype=torch.float64)


In [24]:
print("The Mean Squared Error = \n",loss_fn((X @theta),Rat))

The Mean Squared Error = 
 tensor(0.0044, dtype=torch.float64, grad_fn=<MseLossBackward0>)


In [25]:
print("The sensitivity to the Movie Genres = \n")
print(theta)

The sensitivity to the Movie Genres = 

tensor([[-1.4406, -1.5912, -1.8975],
        [ 0.5945,  2.7722, -0.1911],
        [-0.6395,  1.8339,  0.5603],
        [ 2.6804,  0.7674,  2.7272]], dtype=torch.float64, requires_grad=True)
